### NOTE FOR LUCA

**Remember to set/remove metadata as:**
{
  "nbsphinx": "hidden"
}

to enable/disable solutions view


# Practical 17

In this practical we will keep working with data structures. In particular, we will see a special data structure called *tree*.

## Slides

The slides of the introduction can be found here: [Intro](docs/Practical17.pdf)

## Trees

Trees are data structures composed of two elements *nodes* and *edges*. Nodes represent *things* and edges represent *relationships* among **two** nodes. 

One node called the **root** is the top level of the tree and is connected to one or more other nodes. If the root is connected to another node by means of one edge, then it is said to be the **parent** of the node (and that node is the **child** of the root). Any node can be **parent** of one or more other nodes, the only important thing is that **all nodes have only one parent**. The **root is the only exception as it does not have any parent**. Some nodes do not have children and they are called **leaves**.

An example of graph:


![](img/pract17/tree.png)

As seen in the lecture, graphs can be defined recursively in the following way: *a tree is a root and zero or more subtrees,each of which is also a tree. The root of each subtree is connected to the root of the parent tree by an edge.*.

Graphically: 

![](img/pract17/recursiveTree.png)


Some properties of nodes and trees are:

1. The depth of a node N: the length of the path connecting the root to N counted as number of edges in the path.

2. The level (n) of a tree: set of nodes at the same depth (n).

3. Height : is the maximum depth of all leaves.

4. Width : the maximum number of nodes in each level.

The following tree:

![](img/pract17/exampleTree.png)

has four levels (0,1,2,3), height equal to 3 and width equal to 4.


## ADT: Binary Tree

**Binary trees** are **trees** where each node has at most two children: the **right child** and the **left child**.


The specifications of the binary tree ADT (from the lecture): 

![](img/pract17/binarytreeADT.png)

When implementing a tree we can define a node object and then a tree object that stores nodes, but we will use the more compact way which is to use the recursive definition of a tree.

**Note:** if we want to keep attributes as private, we need to add another method to the previously seen methods: **myTree.setParent(tree)** that sets the parent of a tree **myTree** to **tree**. 

**Example:** Let's implement a binary tree using the recursive definition: a binary tree is a root --with a value --  connected to two subtrees, respectively the right and left subtree. 

In [1]:
%reset -f 

"""This could go to BinaryTree.py"""

class BinaryTree:
    def __init__(self, value):
        self.__data = value
        self.__right = None
        self.__left = None
        self.__parent = None
        
    def getValue(self):
        return self.__data
    
    def setValue(self, newval):
        self.__data = newval
    
    def getParent(self):
        return self.__parent
    #needed because we are using private attributes
    def setParent(self, tree):
        self.__parent = tree
    
    def getRight(self):
        return self.__right
    
    def getLeft(self):
        return self.__left
    
    def insertRight(self, tree):
        if self.__right == None:
            self.__right = tree
            tree.setParent(self) 
                    
    def insertLeft(self, tree):
        if self.__left == None:
            self.__left = tree
            tree.setParent(self)
            
    def deleteRight(self):
        self.__right = None
        
    def deleteLeft(self):
        self.__left = None


    
def printTree(root):
    cur = root
    #each element is a node and a depth
    #depth is used to format prints (with tabs)
    nodes = [(cur,0)]
    tabs = ""
    lev = 0
    while len(nodes) >0:
        cur, lev = nodes.pop(-1)
        #print("{}{}".format("\t"*lev, cur.getValue()))
        if cur.getRight() != None:
            print ("{}{} (r)-> {}".format("\t"*lev, 
                                          cur.getValue(), 
                                          cur.getRight().getValue()))
            nodes.append((cur.getRight(), lev+1))
        if cur.getLeft() != None:
            print ("{}{} (l)-> {}".format("\t"*lev, 
                                          cur.getValue(), 
                                          cur.getLeft().getValue()))
            nodes.append((cur.getLeft(), lev+1))
        
if __name__ == "__main__":
    BT = BinaryTree("Root")
    bt1 = BinaryTree(1)
    bt2 = BinaryTree(2)
    bt3 = BinaryTree(3)
    bt4 = BinaryTree(4)
    bt5 = BinaryTree(5)
    bt6 = BinaryTree(6)
    bt5a = BinaryTree("5a")
    bt5b = BinaryTree("5b")
    bt5c = BinaryTree("5c")
    
    BT.insertLeft(bt1)
    BT.insertRight(bt2)
    bt2.insertLeft(bt3)
    bt3.insertLeft(bt4)
    bt3.insertRight(bt5)
    bt2.insertRight(bt6)
    bt1.insertRight(bt5b)
    bt1.insertLeft(bt5a)
    bt5b.insertRight(bt5c)
    printTree(BT)
    print("\nDelete right branch of 2")
    bt2.deleteRight()
    printTree(BT)
    
    print("\nInsert left branch of 5")
    newN = BinaryTree("child of 5")
    bt5.insertLeft(newN)
    printTree(BT)
   

Root (r)-> 2
Root (l)-> 1
	1 (r)-> 5b
	1 (l)-> 5a
		5b (r)-> 5c
	2 (r)-> 6
	2 (l)-> 3
		3 (r)-> 5
		3 (l)-> 4

Delete right branch of 2
Root (r)-> 2
Root (l)-> 1
	1 (r)-> 5b
	1 (l)-> 5a
		5b (r)-> 5c
	2 (l)-> 3
		3 (r)-> 5
		3 (l)-> 4

Insert left branch of 5
Root (r)-> 2
Root (l)-> 1
	1 (r)-> 5b
	1 (l)-> 5a
		5b (r)-> 5c
	2 (l)-> 3
		3 (r)-> 5
		3 (l)-> 4
			5 (l)-> child of 5


Note that in the code above, deletion of right or left subtree means to just set the pointer to null. This destroys the whole subtree (more clever things can be done in some cases, especially with generic trees). 

Now we can see how to perform some visits of the trees. In particular, there are two different types of visits: **depth first search** and **breadth first search**.

### Binary Tree visits DFS

Given a tree T, depth first search (DFS) visits all the subtrees of T going as deep as it can before going back and down another branch until all the tree is visited.

DFS requires a stack and can be implemented recursively. What we do with the root during the visit defines 3 different types of visits:

1. the root is visited before the visiting the subtree : pre-order;

2. the root is visited after the left subtree but before the right subtree : in-order;

3. the root is visited after the left and right subtrees : post-order 


Consider the following graph:

![](img/pract17/bintree_2.png)


**Example:** Let's implement the three versions of the DFS and apply them to this binary tree.

In [2]:
class BinaryTree:
    def __init__(self, value):
        self.__data = value
        self.__right = None
        self.__left = None
        self.__parent = None
        
    def getValue(self):
        return self.__data
    
    def setValue(self, newval):
        self.__data = newval
    
    def getParent(self):
        return self.__parent
    #needed because we are using private attributes
    def setParent(self, tree):
        self.__parent = tree
    
    def getRight(self):
        return self.__right
    
    def getLeft(self):
        return self.__left
    
    def insertRight(self, tree):
        if self.__right == None:
            self.__right = tree
            tree.setParent(self) 
                    
    def insertLeft(self, tree):
        if self.__left == None:
            self.__left = tree
            tree.setParent(self)
            
    def deleteRight(self):
        self.__right = None
        
    def deleteLeft(self):
        self.__left = None
   
    def preOrderDFS(self):
        if self != None:
            r = self.getRight()
            l = self.getLeft()
            print(self.getValue())
            if l != None:
                l.preOrderDFS()
            if r != None:
                r.preOrderDFS()
    
    def inOrderDFS(self):
        if self != None:
            r = self.getRight()
            l = self.getLeft()
            if l != None:
                l.inOrderDFS()
            print(self.getValue())
            
            if r != None:
                r.inOrderDFS()
            
    def postOrderDFS(self):
        if self != None:
            r = self.getRight()
            l = self.getLeft()
            if l != None:
                l.postOrderDFS()
            if r != None:
                r.postOrderDFS()
            
            print(self.getValue())
    
def printTree(root):
    cur = root
    #each element is a node and a depth
    #depth is used to format prints (with tabs)
    nodes = [(cur,0)]
    tabs = ""
    lev = 0
    while len(nodes) >0:
        cur, lev = nodes.pop(-1)
        #print("{}{}".format("\t"*lev, cur.getValue()))
        if cur.getRight() != None:
            print ("{}{} (r)-> {}".format("\t"*lev, 
                                          cur.getValue(), 
                                          cur.getRight().getValue()))
            nodes.append((cur.getRight(), lev+1))
        if cur.getLeft() != None:
            print ("{}{} (l)-> {}".format("\t"*lev, 
                                          cur.getValue(), 
                                          cur.getLeft().getValue()))
            nodes.append((cur.getLeft(), lev+1))
        
if __name__ == "__main__":
    BT = BinaryTree("Root")
    bt1 = BinaryTree(1)
    bt2 = BinaryTree(2)
    bt3 = BinaryTree(3)
    bt4 = BinaryTree(4)
    bt5 = BinaryTree(5)
    bt6 = BinaryTree(6)
    bt5a = BinaryTree("5a")
    bt5b = BinaryTree("5b")
    bt5c = BinaryTree("5c")
    
    BT.insertLeft(bt1)
    BT.insertRight(bt2)
    bt2.insertLeft(bt3)
    bt3.insertLeft(bt4)
    bt3.insertRight(bt5)
    bt2.insertRight(bt6)
    bt1.insertRight(bt5b)
    bt1.insertLeft(bt5a)
    bt5b.insertRight(bt5c)
    
    printTree(BT)
    print("Pre-order DFS:")
    BT.preOrderDFS()
    print("\nIn-order DFS:")
    BT.inOrderDFS()
    print("\nPost-order DFS:")
    BT.postOrderDFS()

Root (r)-> 2
     (l)-> 1
	1 (r)-> 5b
	  (l)-> 5a
		5b (r)-> 5c
	2 (r)-> 6
	  (l)-> 3
		3 (r)-> 5
		  (l)-> 4
Pre-order DFS:
Root
1
5a
5b
5c
2
3
4
5
6

In-order DFS:
5a
1
5b
5c
Root
4
3
5
2
6

Post-order DFS:
5a
5c
5b
1
4
5
3
6
2
Root


In [3]:
"CODE NOT SHOWN"

#Drawing a graph in pygraphviz
import pygraphviz as pgv

G=pgv.AGraph(directed=True)

#Attributes can be added when adding nodes or edge
G.add_node("Root", color='red')

G.add_node("1", color='blue')
G.add_node("2", color='blue')
G.add_node("3", color='blue')
G.add_node("4", color='black')
G.add_node("5", color='black')
G.add_node("5a", color='black')
G.add_node("5b", color='blue')
G.add_node("5c", color='black')
G.add_edge("Root" ,"2", color='blue')
G.add_edge("Root", "1", color='blue')
G.add_edge("2" ,"3", color='blue')
G.add_edge("2" ,"6", color='blue')
G.add_edge("3" ,"4", color='blue')
G.add_edge("3" ,"5", color='blue')
G.add_edge("1" ,"5a", color='blue')
G.add_edge("1" ,"5b", color='blue')
G.add_edge("5b" ,"5c", color='blue')
# write to a dot file
#G.write('test.dot')

#create a png file
G.layout(prog='dot') # use dot
G.draw('img/pract17/bintree_2.png')

### Binary Tree visits: BFS


### ADT: Generic Tree

Exercise: implement me!


Sets are dynamic data structures that contain **non-repeated** elements in **no specific order**. Sets support the operators: *insert*, *delete* and *contains* to add, remove or test the presence of an element in the set. They have a *minimum* and *maximum* to retrieve the minimum and maximum element (based on values) and it should be possible to iterate through the elements in the set (in no specific order) with something like ```for el in set:```. Finally, some operations are defined on two sets like: *union*, *intersection*, *difference*. 

The specifications of the set ADT (from the lecture):

![](img/pract16/setADT.png)

Note that python natively provides the data structure ```set```. Let's see how to define and interact with it through some examples:

In [4]:
#Drawing a graph in pygraphviz
import pygraphviz as pgv

G=pgv.AGraph(directed=True)

#Attributes can be added when adding nodes or edge
G.add_node("ROOT", color='red')
for i in range(0,3):
    G.add_node("Child_%i" % i, color='blue')
    G.add_node("Grandchild_%ia" % i, color='blue')
    G.add_node("Grandchild_%ib" % i, color='blue')
    G.add_node("Leaf_%i" % i, color='black')

    G.add_edge("ROOT", "Child_%i" % i, color='blue')
    G.add_edge("Child_%i" % i, "Grandchild_%ia" % i, color='blue')
    G.add_edge("Child_%i" % i, "Grandchild_%ib" % i, color='blue')
    if i % 2 == 0:
        G.add_edge("Grandchild_%ia" % i, "Leaf_%i" % i, color='black')        
        G.add_node("Leaf_%ia" %i, color = 'black')
        G.add_node("Leaf_%ib" %i, color = 'black')
        G.add_edge("Grandchild_%ib" % i, "Leaf_%ia" % i, color='black')
        G.add_edge("Grandchild_%ib" % i, "Leaf_%ib" % i, color='black')
    else:
        G.add_edge("Grandchild_%ib" % i, "Leaf_%i" % i, color='black')
        G.add_node("Leaf_%ia" %i, color = 'black')
        G.add_node("Leaf_%ib" %i, color = 'black')
        G.add_edge("Grandchild_%ia" % i, "Leaf_%ia" % i, color='black')
        G.add_edge("Grandchild_%ia" % i, "Leaf_%ib" % i, color='black')

# write to a dot file
#G.write('test.dot')

#create a png file
G.layout(prog='dot') # use dot
G.draw('img/pract17/tree.png')





In [5]:
"""set examples"""

#empty set
a = set()
print(a)
a.add("Luca")
a.add("Alberto")
a.add("David")
print(a) 
#adding the same element twice...
a.add("Luca")
#..has no effect
print(a)

#a set from a list of values
myL = [121,5,4,1,1,4,2,121]
print("\nList:{}".format(myL))
S = set(myL)
print("Set:{}".format(S))

#accessing elements:
for el in S:
    print("\telement: {}".format(el))

print("44 in S? {}".format(44 in S))
print("121 in S? {}".format(121 in S))

#from strings
S1 = set("abracadabra")
S2 = set("AbbadabE")
print("\nS1:{}".format(S1))
print("S2:{}".format(S2))
print("\nIntersection(S1,S2): {}".format(S1 & S2))
print("\nUnion(S1,S2): {}".format(S1 | S2))
print("\nIn S1 but not in S2: {}".format(S1 - S2))
print("In S2 but not in S1: {}".format(S2 - S1))
print("\nIn S1 or S2 but not in both: {}".format(S1 ^ S2))


set()
{'Luca', 'Alberto', 'David'}
{'Luca', 'Alberto', 'David'}

List:[121, 5, 4, 1, 1, 4, 2, 121]
Set:{121, 2, 4, 5, 1}
	element: 121
	element: 2
	element: 4
	element: 5
	element: 1
44 in S? False
121 in S? True

S1:{'a', 'r', 'd', 'c', 'b'}
S2:{'E', 'A', 'd', 'b', 'a'}

Intersection(S1,S2): {'a', 'd', 'b'}

Union(S1,S2): {'A', 'r', 'c', 'b', 'E', 'a', 'd'}

In S1 but not in S2: {'r', 'c'}
In S2 but not in S1: {'E', 'A'}

In S1 or S2 but not in both: {'A', 'r', 'c', 'E'}


## Collections

Python provides several data structures in the ```collections``` module, like for example, **dequeues** etc. You can find more information at [https://docs.python.org/3.5/library/collections.html?highlight=collections#module-collections](https://docs.python.org/3.5/library/collections.html?highlight=collections#module-collections).



## Iterating through a custom collection

To be able to iterate through the elements of our **custom** collection with something like **for x in mycollection.iterate()**, we need a way to generate one element at a time until all elements have been obtained. That is, we need to define an *iterator*, which is a procedure that returns all the elements one at a time, when asked by a caller. Python provides the keyword ```yield``` for this. Let's see how this works with an example.

**Example:** Given a myClass class that stores three lists of 3D coordinates (assumed to have the same size), let's define an iterator that returns each triplet one after the other.

In [6]:
class myClass:
    def __init__(self, x,y,z):
        self.__x = x
        self._y = y
        self._z = z
        
    def add(self,x,y,z):
        self.__x.extend(x)
        self._y.extend(y)
        self._z.extend(z)
        
    def iterator(self):
        for i in range(len(self.__x)):
            yield (self.__x[i], self._y[i], self._z[i])


C = myClass([0,1,2,3,4], [0,1,0,1,0], [4,3,2,1,0])
C.add([27,44],[14,4],[27,1])
for el in C.iterator():
    print("Element: {}".format(el))

print(C.__x)


Element: (0, 0, 4)
Element: (1, 1, 3)
Element: (2, 0, 2)
Element: (3, 1, 1)
Element: (4, 0, 0)
Element: (27, 14, 27)
Element: (44, 4, 1)


AttributeError: 'myClass' object has no attribute '__x'

## Exercises


1. Implement delete node of a tree

2. Modify the original version adding a depth parameter to the binary node. Implement get width etc page 34

3. Implement get common ancestor

3. Implement Binary search tree

3. Implement the generic tree ADT


The ADT of the set structure is (i.e. the methods to implement): 

![](img/pract16/setADT.png)

Implement a **iterator** method that **yields** the next elements. Implement a special method "__contains__" to test if an element is present with **el in S**. 

Test the code with:

```
    S = MySet([33, 1,4,5,7,5,5,5,4,7,3])
    print("Initial S: {}".format(S))
    S.add(125)
    S.discard(77)
    S.discard(5)
    print("S now: {}".format(S))
    print("Does S contain 13? {}".format(13 in S))
    print("Does S contain 125? {}".format(125 in S))
    print("All elements in S:")
    for s in S.iterator():
        print("\telement: {}".format(s))

    print("\nS:{}".format(S))
    S1 = MySet([33, 0, 3,4, 4, 33,44])
    print("S1: {}".format(S1))
    print("\nUnion: {}".format(S.union(S1)))
    print("Intersection: {}".format(S.intersection(S1)))
    print("S - S1: {}".format(S.difference(S1)))
    print("S1 - S: {}".format(S1.difference(S)))
    print("(S - S1) U (S1 -S): {}".format(S.difference(S1).union(S1.difference(S))))
```

and compare the results with what would give python's built-in ```set``` data structure.


<div class="tggle" onclick="toggleVisibility('ex1');">Show/Hide Solution</div>
<div id="ex1" style="display:none;">

In [ ]:
%reset -f


class MySet:
    def __init__(self, elements):
        self.__data = dict()
        for el in elements:
            self.__data[el] = 1
    
    #let's specify the special operator for len
    def __len__(self):
        return len(self.__data)
    
    #this is the special operator for in
    def __contains__(self, element):
        el = self.__data.get(element, None)
        if el != None:
            return True
        else:
            return False
    
    #we do not redefine __add_ because that is for S1 + S2
    #where S1 and S2 are sets
    def add(self,element):
        #dont care if already there
        self.__data[element] = 1 
    
    def discard(self,element):
        #equivalent to: 
        #if element in self.__data: del self.__data[element]
        el = self.__data.pop(element, None)
    
    def iterator(self):
        keys = list(self.__data.keys())
        for i in range(len(keys)):
            yield keys[i]
            
    def __str__(self):
        keys = self.__data.keys() 
        return "{"+"{}".format(", ".join([str(x) for x in keys])) + "}"

    def union(self, other):
        """elements in either of the two sets"""
        elements = []
        for el in other.iterator():
            elements.append(el)
        S = MySet(elements)
        
        for el in self.iterator():
            S.add(el)
        return S
    def intersection(self, other):
        """elements in both sets"""
        inter = [x for x in self.iterator() if x in other.iterator()]
        return MySet(inter)

    def difference(self, other):
        """elements in self but not in other"""
        diff = [x for x in self.iterator() if x not in other.iterator()]
        return MySet(diff)

if __name__ == "__main__":
    
    S = MySet([33, 1,4,5,7,5,5,5,4,7,3])
    print("Initial S: {}".format(S))
    S.add(125)
    S.discard(77)
    S.discard(5)
    print("S now: {}".format(S))
    print("Does S contain 13? {}".format(13 in S))
    print("Does S contain 125? {}".format(125 in S))
    print("All elements in S:")
    for s in S.iterator():
        print("\telement: {}".format(s))

    print("\nS:{}".format(S))
    S1 = MySet([33, 0, 3,4, 4, 33,44])
    print("S1: {}".format(S1))
    print("\nUnion: {}".format(S.union(S1)))
    print("Intersection: {}".format(S.intersection(S1)))
    print("S - S1: {}".format(S.difference(S1)))
    print("S1 - S: {}".format(S1.difference(S)))
    print("(S - S1) U (S1 -S): {}".format(S.difference(S1).union(S1.difference(S))))
    
    #### Test vs python's set
    print("Testing python's builtin:")
    pS = set([33, 1,4,5,7,5,5,5,4,7,3])
    pS.add(125)
    #pS.remove(77)
    pS.remove(5)
    print("pS: {}".format(pS))
    pS1 = set([33, 0, 3,4, 4, 33,44])
    print("pS1: {}".format(pS1))
    print("Union: {}".format(pS | pS1))
    print("Intersection: {}".format(pS & pS1))
    print("pS - pS1: {}".format(pS - pS1))
    print("pS1 - pS: {}".format(pS1 - pS))
    print("(pS - pS1) U (pS1 -pS): {}".format(pS - pS1 | pS1 - pS))

</div>

2. Complete the bidirectional linked list example seen above by adding a ```remove(x)``` method that removes element x from the list if present and a ```slice(x,y)``` where ```x``` and ```y``` are integers. The slice method should return another bidirectionaly linked list with elements from ```x``` (included) to ```y``` (excluded). 



<div class="tggle" onclick="toggleVisibility('ex2');">Show/Hide Solution</div>
<div id="ex2" style="display:none;">

In [ ]:
%reset -f 

""" Can place this in Node.py"""
class Node:
    def __init__(self, data):
        self.__data = data
        self.__prevEl = None
        self.__nextEl = None
    
    def getData(self):
        return self.__data
    
    def setData(self, newdata):
        self.__data = newdata
    
    def setNext(self, node):
        self.__nextEl = node
    
    def getNext(self):
        return self.__nextEl
    
    def setPrev(self,node):
        self.__prevEl = node
    def getPrev(self):
        return self.__prevEl
    
    def __str__(self):
        return str(self.__data)
    #for sorting
    def __lt__(self, other):
        return self.__data < other.__data
    
        
""" Can place this in BiLinkList.py"""        
class BiLinkList:
    def __init__(self):
        self.__head = None
        self.__tail = None
        self.__len = 0
        self.__minEl = None
        self.__maxEl = None
        
    def __len__(self):
        return self.__len
    def min(self):
        return self.__minEl
    def max(self):
        return self.__maxEl
    
    def append(self,node):
        if type(node) != Node:
            raise TypeError("node is not of type Node")
        else:
            if self.__head == None:
                self.__head = node
                self.__tail = node
            else:
                node.setPrev(self.__tail)
                self.__tail.setNext(node)
                self.__tail = node
                
            self.__len += 1
            #This assumes that nodes can be sorted
            if self.__minEl == None or self.__minEl > node:
                self.__minEl = node
            if self.__maxEl == None or self.__maxEl < node:
                self.__maxEl = node
                
    def insert(self, node, i):
        # to avoid index problems, if i is out of bounds
        # we insert at beginning or end
        if i > self.__len:
            i = self.__len #I know that it is after tail!
        if i < 0:
            i = 0
        cnt = 0
        cur_el = self.__head
        while cnt < i:
            cur_el = cur_el.getNext()
            cnt += 1
        #add node before cur_el
        if cur_el == self.__head:
            #add before current head
            node.setNext(self.__head)
            self.__head.setPrev(node)
            self.__head = node
        else:
            if cur_el == None:
                #add after tail
                self.__tail.setNext(node)
                node.setPrev(self.__tail)
                self.__tail = node
            else:
                p = cur_el.getPrev()
                p.setNext(node)
                node.setPrev(p)
                node.setNext(cur_el)
                cur_el.setPrev(node)
        
        self.__len += 1
        #This assumes that nodes can be sorted
        if self.__minEl == None or self.__minEl > node:
            self.__minEl = node
        if self.__maxEl == None or self.__maxEl < node:
            self.__maxEl = node
    
    def getAtIndex(self, i):
        if i > self.__len:
            return None
        else:
            cnt = 0
            cur_el = self.__head
            while cnt < self.__len:
                if cnt == i:
                    return cur_el
                else:
                    cnt += 1
                    cur_el = cur_el.getNext()
    
    def iterator(self):
        cur_el = self.__head
        while cur_el != None:
            yield cur_el
            cur_el = cur_el.getNext()
    
    def __str__(self):
        
        if self.__head != None:
            dta = str(self.__head)
            cur_el = self.__head.getNext() 
            while cur_el != None:
                dta += " <-> " + str(cur_el)
                cur_el = cur_el.getNext()

            return str(dta)
        else:
            return ""
    
    ###################################################
    ################### NEW METHODS ###################
    ###################################################
    def remove(self, element):
        if self.__head != None:
            cur_el = self.__head
            while cur_el != element and cur_el != None:
                cur_el = cur_el.getNext()

            if cur_el != None:
                p = cur_el.getPrev()
                n = cur_el.getNext()

                if cur_el == self.__head:
                    self.__head = n

                if cur_el == self.__tail:
                    self.__tail = p

                if n != None:
                    n.setPrev(p)
                if p != None:
                    p.setNext(n)
                    
                self.__len -= 1

            
    def slice(self, x, y):
        m = min(x,y)
        M = max(x,y)
        
        if m > self.__len:
            return None
        else:
            cur_el = self.__head
            cnt = 0
            while cnt < m:
                cur_el = cur_el.getNext()
                cnt += 1
            nList = BiLinkList()
            
            while cnt < M and cur_el != None:
                    n = Node(cur_el.getData())
                    cur_el = cur_el.getNext()
                    nList.append(n)
                    cnt += 1
            return nList
    
    ###################################################
    ############### END NEW METHODS ###################
    ###################################################

if __name__ == "__main__":
    import random
    MLL = BiLinkList()
    for i in range(1,50,10):
        n = Node(i)
        MLL.append(n)
    print(MLL)
    for el in MLL.iterator():
        print("\t{} prev:{} next:{}".format(el,el.getPrev(), 
                                            el.getNext()))
    n = Node(2)
    MLL.insert(n,2)
    n = Node(-10)
    MLL.append(n)
    n = Node(1000)
    MLL.insert(n, -1)
    n = Node(27)
    MLL.insert(n, 2000)
    
    print(MLL)
    for el in MLL.iterator():
        print("\t{} prev:{} next:{}".format(el,el.getPrev(), 
                                            el.getNext()))
    print("Number of elements: {} min: {}  max: {}".format(len(MLL),
                                                           MLL.min(), 
                                                           MLL.max()))
    n = MLL.getAtIndex(3)
    print("MLL[3] = {}".format(n))
    MLL.remove(n)
    print("{} removed!".format(n))
    print(MLL)
    for el in MLL.iterator():
        print("\t{} prev:{} next:{}".format(el,el.getPrev(), 
                                            el.getNext()))
    
    n = MLL.getAtIndex(0)
    print("MLL[0] = {}".format(n))
    MLL.remove(n)
    print("{} removed!".format(n))
    print(MLL)
    
    for el in MLL.iterator():
        print("\t{} prev:{} next:{}".format(el,el.getPrev(), 
                                            el.getNext()))
        
    #slice:
    print("Slice[2,4]:")
    print(MLL.slice(2,4))

    #slice:
    print("Slice[3,15]:")
    print(MLL.slice(3,15))
    
    #Removing all elements now.
    print("Remove all")
    for i in range(len(MLL)):
        n = MLL.getAtIndex(0)
        MLL.remove(n)
        print("{} removed!".format(n))
        print(MLL)
    print(MLL)

</div>

3. Stacks are great to evaluate postfix expressions. Some examples of postfix expressions are:

```
10 5 +
```
that encodes for ```10 + 5 = 15``` 

``` 10 5  + 7 *```
that encodes for ```(10 + 5) * 7 = 105```

Given a postfix expression it can be evaluated in the following way: 

1. start from the beginning of the string (better, list obtained by splitting by " "), remove the first element and insert elements in the stack unless they are operators. If they are operators, pop two elements and apply the operation, storing it in a varible;

2. If the list is empty, the result is stored in the variable, otherwise go back to point 1.

Assuming only integer numbers and the 4 standard binary operators +,-,/,\*: write some python code that uses the stack class seen above ```MyStack``` and evaluates the following postfix expressions:

```
operations = ["10 5  + 7 *", "1 2 3 4 5 6 7 8 + + + + + + +", "1 2 3 4 5 + - * /"]

```
<div class="tggle" onclick="toggleVisibility('ex3');">Show/Hide Solution</div>
<div id="ex3" style="display:none;">

In [ ]:
%reset -f

class MyStack:
    
    def __init__(self):
        self.__data = []
    
    def isEmpty(self):
        return len(self.__data) == 0
    
    def __len__(self):
        return len(self.__data)
    
    def push(self, element):
        """adds an element on top of the stack"""
        self.__data.append(element)
        
    def pop(self):
        """removes one element from the stack and returns it"""
        if len(self.__data) > 0:
            ret = self.__data[-1]
            del self.__data[-1]
            return ret
        else:
            return None
    
    def peek(self):
        if len(self.__data) > 0:
            return self.__data[-1]
        else:
            return None
        

def evaluatePostfix(expr):        
    S = MyStack()
    els = expr.split(" ")
    res = 0
    infix = ""
    for i in range(len(els)):
        e = els[i]
        if e not in "+-*/":
            S.push(int(e))
        else:
            o2 = S.pop()
            o1 = S.pop()
            tmp = 0
            infix = "(" + str(o1) + " " +e +" " + str(o2) + ")"  
            print(infix)
            if e == "+":
                tmp = o1 + o2
            
            elif e == "-":
                tmp = o1 - o2
            
            elif e == "/":
                tmp = o1 / o2
            
            else:
                tmp = o1 * o2
            res = tmp
            
            
            if i != len(els):
                S.push(res)
    return res


operations = ["10 5 + 7 *", 
              "1 2 3 4 5 6 7 8 + + + + + + +", 
              "1 2 3 4 5 + - * /",
             "5 4 + 8 /",
             "3 10 2 - 5 * +"]

for op in operations:
    print("Operation: {}".format(op))
    res = evaluatePostfix(op)
    print("Result: {}".format(res))
    

</div>

4. Implement a circular single-directional linked list of objects SingleNode (that have a data and a link to the next element) with the following methods: 

a. append(element) : adds at the end of the list;

b. extend(list_of_elements) : adds all the elements in the list

c. get(index) : reads the node at position index (if index is lower than length else return None);

d. removeAt(index) : removes the element at position index if it exists;

e. removeEl(el) : removes the element el, if present.

f. head() : gets the first element of the list;

g. tail() : gets the last element of the list;

h. __len__() : returns the length of the list;

i. __str__() : returns a string representation of the list: 

```
1 --> 2 --> 3 --> ... N --|
^-------------------------|  
```

Remember that a circular list should always have the last element (tail) pointing to the first element (head):

![](img/pract16/circular_list.png)

Test your class with the following code:

```
    CL = CircularList()
    n = SingleNode([1])
    n1 = SingleNode(2)
    n2 = SingleNode([3])
    n3 = SingleNode([4])
    n4 = SingleNode(5)
    n5 = SingleNode([6])
    CL.append(n)
    CL.append(n1)
    CL.append(n2)
    CL.extend([n3,n4,n5])
    n = SingleNode("luca")
    CL.append(n)
    print(CL)
    print("CL has length: {}".format(len(CL)))
    print("Head:{}\nTail:{}".format(CL.head(),CL.tail()))
    print("{} is at position: {}".format(CL.get(3),3))
    print("{} is at position: {}".format(CL.get(-10),-10))
    print("{} is at position: {}".format(CL.get(20),20))
    print("{} is at position: {}".format(CL.get(0),0))
    CL.removeAt(2)
    CL.removeAt(5)
    print(CL)
    CL.removeEl(n5)
    print(CL)
    #n is not present!
    CL.removeEl(n)
    print(CL)
```

<div class="tggle" onclick="toggleVisibility('ex4');">Show/Hide Solution</div>
<div id="ex4" style="display:none;">

In [ ]:
%reset -f 

""" Can place this in SingleNode.py"""
class SingleNode:
    def __init__(self, data):
        self.__data = data
        self.__nextEl = None
    
    def getData(self):
        return self.__data
    
    def setData(self, newdata):
        self.__data = newdata
    
    def setNext(self, node):
        self.__nextEl = node
    
    def getNext(self):
        return self.__nextEl
    
    
    def __str__(self):
        return str(self.__data)
    #for sorting
    def __lt__(self, other):
        return self.__data < other.__data
    
    
"""Can place this in CircularList.py"""
class CircularList:
    def __init__(self):
        self.__head = None
        self.__tail = None
        self.__len = 0
    
    def __len__(self):
        return self.__len
    
    def append(self, node):
        if type(node) != SingleNode:
            raise TypeError("node is not of type Node")
        else:
            if self.__head == None:
                self.__head = node
                self.__tail = node
            else:
                node.setNext(self.__head)
                self.__tail.setNext(node)
                self.__tail = node
                
            self.__len += 1
        
    def extend(self, nodesList):
        for el in nodesList:
            self.append(el)
            
    def head(self):
        return self.__head
    
    def tail(self):
        return self.__tail
    
    def get(self, index):
        i = 0
        cur_el = self.__head
        if index < 0:
            #should someone input a very small number!
            while index < 0:
                index = self.__len + index 
        
        while i < index:
            cur_el = cur_el.getNext()
            i += 1
        return cur_el
        
    
    def removeAt(self, index):
        i = 0
        cur_el = self.__head
        if index < 0:
            #should someone input a very small number!
            while index < 0:
                index = self.__len + index 
            
        
        while i < index-1:
            cur_el = cur_el.getNext()
            i += 1
        prev = cur_el
        cur_el = prev.getNext()
        next_el = cur_el.getNext()
        prev.setNext(next_el)
        if cur_el == self.__tail:
            self.__tail = prev
        if cur_el == self.__head:
            self.__head = prev
            
        self.__len -= 1
    
    def removeEl(self, element):
        i = 0
        cur_el = self.__head
        
        while cur_el.getNext() != element and cur_el != self.__tail:
            cur_el = cur_el.getNext()
            
        if cur_el != self.__tail: 
            prev = cur_el
            cur_el = prev.getNext()
            #cur_el is element now
            next_el = cur_el.getNext()
            prev.setNext(next_el)
            if cur_el == self.__tail:
                self.__tail = prev
            if cur_el == self.__head:
                self.__head = prev

            self.__len -= 1
        
    def __str__(self):
        outStr = ""
        cur_el = self.__head
        outStr = str(cur_el)
        while cur_el != self.__tail:
            cur_el = cur_el.getNext()
            outStr += "-->" + str(cur_el)
        L = len(outStr)
        outStr += "--|\n^"
        

    
        i = 0
        while i < L+1:
            outStr = outStr + "-"
            i += 1 
        outStr += "|"
        
        return outStr

if __name__ == "__main__":
    CL = CircularList()
    n = SingleNode([1])
    n1 = SingleNode(2)
    n2 = SingleNode([3])
    n3 = SingleNode([4])
    n4 = SingleNode(5)
    n5 = SingleNode([6])
    CL.append(n)
    CL.append(n1)
    CL.append(n2)
    CL.extend([n3,n4,n5])
    n = SingleNode("luca")
    CL.append(n)
    print(CL)
    print("CL has length: {}".format(len(CL)))
    print("Head:{}\nTail:{}".format(CL.head(),CL.tail()))
    print("{} is at position: {}".format(CL.get(3),3))
    print("{} is at position: {}".format(CL.get(-10),-10))
    print("{} is at position: {}".format(CL.get(20),20))
    print("{} is at position: {}".format(CL.get(0),0))
    CL.removeAt(2)
    CL.removeAt(5)
    print(CL)
    CL.removeEl(n5)
    print(CL)
    #n is not present!
    CL.removeEl(n)
    print(CL)
    


</div>

(exercise4) monopoly as linked list. Pick random numbers from 1 to 6 (money, bankrupt)
